In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 27.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatib

In [20]:
from ortools.sat.python import cp_model
import random

NUM_STUDENTS = 200
NUM_CLASSROOMS = 6
CLASSROOM_IDS = list(range(NUM_CLASSROOMS))
STUDENT_PER_CLASSROOM = [33, 33, 33, 33, 34, 34]

import pandas as pd

df = pd.read_csv('/content/학급반편성CSP 문제 입력파일.csv')
df['id'] = df['id'].astype(int)


model = cp_model.CpModel()

# 기본 세팅
student_vars = {}
for sid in df['id']:
    student_vars[int(sid)] = model.NewIntVar(0, NUM_CLASSROOMS - 1, f"student_{int(sid)}_class")

assigned_to_class = {}
for sid in df['id']:
    s = int(sid)
    assigned_to_class[s] = {}
    bools = []
    for c_id in CLASSROOM_IDS:
        b = model.NewBoolVar(f"student_{s}_in_class_{c_id}")
        assigned_to_class[s][c_id] = b
        bools.append(b)
        model.Add(student_vars[s] == c_id).OnlyEnforceIf(b)
        model.Add(student_vars[s] != c_id).OnlyEnforceIf(b.Not())
    model.AddExactlyOne(bools)


# ✅ 각 클래스에 배정되어야 하는 학생 수
for c_id in CLASSROOM_IDS:
    students_in_this_class = [assigned_to_class[int(s_id)][c_id] for s_id in df['id']]
    required = STUDENT_PER_CLASSROOM[c_id]
    model.Add(sum(students_in_this_class) == required)



# ✅ 각 교실에 리더십 학생 최소 한명 이상
leadership_students = df[df['Leadership'] == 'yes']['id'].tolist()

for c_id in CLASSROOM_IDS:
        leadership_vars_in_class = [
            assigned_to_class[s_id][c_id]
            for s_id in leadership_students
        ]
        model.Add(sum(leadership_vars_in_class) >= 1)

# ✅ 피아노 연주 가능한 학생 균등 분배
piano_students = df[df["Piano"].astype(str).str.strip().str.lower() == "yes"]["id"].astype(int).tolist()
num_piano = len(piano_students)
avg_piano_per_class = num_piano / NUM_CLASSROOMS

for c_id in CLASSROOM_IDS:
    piano_vars_in_class = [assigned_to_class[s_id][c_id] for s_id in piano_students]
    model.Add(sum(piano_vars_in_class) >= int(avg_piano_per_class) - 2)
    model.Add(sum(piano_vars_in_class) <= int(avg_piano_per_class) + 2)

# ✅  남녀 비율 제약조건
boys = df[df["sex"].str.lower().str.strip() == "boy"]["id"].astype(int).tolist()
girls = df[df["sex"].str.lower().str.strip() == "girl"]["id"].astype(int).tolist()
total_boys = len(boys)
expected_boys_per_class = total_boys / NUM_CLASSROOMS

for c_id in CLASSROOM_IDS:
    boys_in_class = [assigned_to_class[s_id][c_id] for s_id in boys]
    model.Add(sum(boys_in_class) >= int(expected_boys_per_class) - 2)
    model.Add(sum(boys_in_class) <= int(expected_boys_per_class) + 2)


# ✅ 운동 능력 균등 배분
exercise_students = df[df["운동선호"].astype(str).str.strip().str.lower() == "yes"]["id"].astype(int).tolist()
num_exercise = len(exercise_students)
avg_exercise_per_class = num_exercise / NUM_CLASSROOMS

for c_id in CLASSROOM_IDS:
    exercise_vars_in_class = [assigned_to_class[s_id][c_id] for s_id in exercise_students]
    model.Add(sum(exercise_vars_in_class) >= int(avg_exercise_per_class) - 2)
    model.Add(sum(exercise_vars_in_class) <= int(avg_exercise_per_class) + 2)



# ✅  클럽별 균등 분배 제약
clubs = df["클럽"].dropna().unique().tolist()
for club_name in clubs:
    club_students = df[df["클럽"] == club_name]["id"].astype(int).tolist()
    num_club = len(club_students)
    avg_per_class = num_club / NUM_CLASSROOMS

    for c_id in CLASSROOM_IDS:
        club_vars_in_class = [assigned_to_class[s_id][c_id] for s_id in club_students]
        model.Add(sum(club_vars_in_class) >= int(avg_per_class) - 2)
        model.Add(sum(club_vars_in_class) <= int(avg_per_class) + 2)

# ✅ 같은 클래스 문제 아동학생 피하는 제약
student_dislikes = []

for idx, row in df.iterrows():
  if not pd.isnull(row["나쁜관계"]):
    std1, std2 = row["id"], int(row["나쁜관계"])
    student_dislikes.append([std1, std2])

dislike_penalties = []
for s1, s2 in student_dislikes:
    pair_overlap = model.NewBoolVar(f"dislike_overlap_{s1}_{s2}")

    or_list = []
    for c_id in CLASSROOM_IDS:
        both_in_c = model.NewBoolVar(f"both_{s1}_{s2}_in_{c_id}")
        model.AddBoolAnd([assigned_to_class[s1][c_id], assigned_to_class[s2][c_id]]).OnlyEnforceIf(both_in_c)
        model.AddBoolOr([assigned_to_class[s1][c_id].Not(), assigned_to_class[s2][c_id].Not()]).OnlyEnforceIf(both_in_c.Not())
        or_list.append(both_in_c)

    model.AddBoolOr(or_list).OnlyEnforceIf(pair_overlap)
    model.AddBoolAnd([oc.Not() for oc in or_list]).OnlyEnforceIf(pair_overlap.Not())

    dislike_penalties.append(pair_overlap)

model.Minimize(sum(dislike_penalties))


# ✅  전년도 같은 반 겹침 최소화
prev_classes = df["24년 학급"].dropna().unique().tolist()

prev_class_counts_vars = []

for c_id in CLASSROOM_IDS:
    students_in_class = [assigned_to_class[s_id][c_id] for s_id in df['id'].astype(int)]

    for prev_cls in prev_classes:
        prev_cls_students = df[df["24년 학급"] == prev_cls]["id"].astype(int).tolist()
        if len(prev_cls_students) == 0:
            continue

        bool_vars = [assigned_to_class[s_id][c_id] for s_id in prev_cls_students]

        count_var = model.NewIntVar(0, len(prev_cls_students), f"class_{c_id}_prev_{prev_cls}_count")
        model.Add(count_var == sum(bool_vars))

        prev_class_counts_vars.append(count_var)

model.Minimize(sum(prev_class_counts_vars))


# ✅ 성적, 학력 순으로 분배
class_score_sums = []
for c_id in CLASSROOM_IDS:
    score_sum_var = model.NewIntVar(0, int(df['score'].max()*NUM_STUDENTS), f"class_{c_id}_score_sum")

    model.Add(score_sum_var == sum(int(df.loc[df['id']==s_id, 'score'].values[0]) * assigned_to_class[s_id][c_id] for s_id in df['id']))
    class_score_sums.append(score_sum_var)

score_order_violations = []
for i in range(NUM_CLASSROOMS-1):
    violation = model.NewBoolVar(f"score_violation_{i}")
    model.Add(class_score_sums[i] < class_score_sums[i+1]).OnlyEnforceIf(violation)
    model.Add(class_score_sums[i] >= class_score_sums[i+1]).OnlyEnforceIf(violation.Not())
    score_order_violations.append(violation)


# ✅ "비등교" 학생 균등 분배
non_attend_students = df[df["비등교"].astype(str).str.strip().str.lower() == "yes"]["id"].astype(int).tolist()
num_non_attend = len(non_attend_students)
avg_non_attend_per_class = num_non_attend / NUM_CLASSROOMS
tolerance = 1  # ± 허용치

for c_id in CLASSROOM_IDS:
    non_attend_vars_in_class = [assigned_to_class[s_id][c_id] for s_id in non_attend_students]

    model.Add(sum(non_attend_vars_in_class) >= int(avg_non_attend_per_class) - tolerance)
    model.Add(sum(non_attend_vars_in_class) <= int(avg_non_attend_per_class) + tolerance)



import random

# ✅  비등교 - 일반 학생 담당 배정
non_attend_students = df[df["비등교"].astype(str).str.strip().str.lower() == "yes"]["id"].astype(int).tolist()
attend_students = df[df["비등교"].isna()]["id"].astype(int).tolist()

random.shuffle(attend_students)
pairs = list(zip(non_attend_students, attend_students[:len(non_attend_students)]))

pair_together_vars = []
for s1, s2 in pairs:
    same_class_var = model.NewBoolVar(f"pair_{s1}_{s2}_together")

    or_list = []
    for c_id in CLASSROOM_IDS:
        both_in_c = model.NewBoolVar(f"both_{s1}_{s2}_in_{c_id}")
        model.AddBoolAnd([assigned_to_class[s1][c_id], assigned_to_class[s2][c_id]]).OnlyEnforceIf(both_in_c)
        model.AddBoolOr([assigned_to_class[s1][c_id].Not(), assigned_to_class[s2][c_id].Not()]).OnlyEnforceIf(both_in_c.Not())
        or_list.append(both_in_c)

    model.AddBoolOr(or_list).OnlyEnforceIf(same_class_var)
    model.AddBoolAnd([oc.Not() for oc in or_list]).OnlyEnforceIf(same_class_var.Not())

    pair_together_vars.append(same_class_var)

model.Maximize(sum(pair_together_vars))



model.Minimize(sum(prev_class_counts_vars) + sum(dislike_penalties) + sum(score_order_violations) - sum(pair_together_vars))


# --- 솔버 설정 및 실행 ---
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 600.0   # 120초(2분)
solver.parameters.num_workers = 8

print("모델을 풀고 있습니다...")
status = solver.Solve(model)

# --- 결과 출력 ---
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"✅ 해결책 발견 (상태: {solver.StatusName(status)})")
    print(f"  - 실제 시간: {solver.WallTime():.2f}초")

    # 교실별 배정 결과
    classroom_assignments = {c_id: [] for c_id in CLASSROOM_IDS}
    for s_id in df['id'].astype(int):
        assigned_class = solver.Value(student_vars[s_id])
        classroom_assignments[assigned_class].append(s_id)

    print(f"\n✨ 최종 배정 결과 (총 학생 수: {NUM_STUDENTS}, 총 교실 수: {NUM_CLASSROOMS})")
    for c_id in CLASSROOM_IDS:
        students_in_class = classroom_assignments[c_id]

        leadership_count = sum(1 for s in students_in_class if int(s) in leadership_students)
        capacity_status = "✅ 정원 만족" if len(students_in_class) == STUDENT_PER_CLASSROOM[c_id] else "❌ 정원 불만족"
        leadership_status = "✅ 리더십 ≥ 1" if leadership_count >= 1 else "⚠️ 리더십 부족 (0명)"
        piano_count = sum(1 for s in students_in_class if int(s) in piano_students)

        # 남녀 수 계산
        boys_in_class = sum(1 for s in students_in_class if int(s) in boys)
        girls_in_class = sum(1 for s in students_in_class if int(s) in girls)
        total_in_class = len(students_in_class)

        # 성비 계산 (0으로 나누는 오류 방지)
        boy_ratio = boys_in_class / total_in_class if total_in_class > 0 else 0
        girl_ratio = girls_in_class / total_in_class if total_in_class > 0 else 0

        # 운동
        exercise_count = sum(1 for s in students_in_class if int(s) in exercise_students)

        # 각 반의 클럽 분포 출력
        club_counts = {}
        for club_name in clubs:
            count = sum(1 for s in students_in_class if df.loc[df["id"] == s, "클럽"].iloc[0] == club_name)
            club_counts[club_name] = count


        # 싫어하는 관계의 학생 수
        overlap_dislike_pairs = 0
        for s1, s2 in student_dislikes:
            if s1 in students_in_class and s2 in students_in_class:
                overlap_dislike_pairs += 1

        # 새 학급 안에서 전년도 학급별 학생 수 세기
        prev_class_counts = []
        for prev_cls in df["24년 학급"].dropna().unique():
            count = sum(1 for s_id in students_in_class if df.loc[df['id'] == s_id, "24년 학급"].values[0] == prev_cls)
            prev_class_counts.append(count)
        avg_overlap = sum(prev_class_counts) / len(prev_class_counts) if len(prev_class_counts) > 0 else 0

        # 학급 별 평균 점수
        students_in_class = [int(s) for s in classroom_assignments[c_id]]
        if len(students_in_class) == 0:
            avg_score = 0
        else:
            avg_score = sum(df.loc[df['id'] == s_id, 'score'].values[0] for s_id in students_in_class) / len(students_in_class)

        students_in_class = [int(s) for s in classroom_assignments[c_id]]
        non_attend_count = sum(1 for s_id in students_in_class if s_id in non_attend_students)

         # 비등교-일반 학생 매칭 쌍 수
        non_attend_pairs_in_class = 0
        for s1, s2 in pairs:  # 이전에 만든 pairs 리스트 사용
            if s1 in students_in_class and s2 in students_in_class:
                non_attend_pairs_in_class += 1



        print(f"\n[교실 {c_id}] (필요 정원: {STUDENT_PER_CLASSROOM[c_id]})")
        print(f"  - 배정 학생 수: {len(students_in_class)} ({capacity_status})")
        print(f"  - 리더십 학생 수: {leadership_count}명 ({leadership_status})")
        print(f"  - 피아노 연주 학생 수: {piano_count}명")
        print(f"  - 남학생: {boys_in_class}명, 여학생: {girls_in_class}명")
        print(f"  - 성비: 남 {boy_ratio:.2%} / 여 {girl_ratio:.2%}")
        print(f"  - 운동 선호 학생 수: {exercise_count}명")
        print(f"  - 클럽 분포: {club_counts}")
        print(f"  - 싫어하는 관계 쌍 수: {overlap_dislike_pairs}")
        print(f"  - 교실 {c_id}: 전년도 같은 반 학생 수 평균 = {avg_overlap:.2f}")
        print(f"  - 교실 {c_id}: 평균 점수 = {avg_score:.2f}")
        print(f"  - 교실 {c_id} 비등교 학생 수: {non_attend_count}명")
        print(f"  - 교실 {c_id}: 비등교-일반 학생 매칭 쌍 수 = {non_attend_pairs_in_class}쌍")


        # print(f"  - 전년도 같은 반 학생 쌍 겹침 수: {overlapping_pairs}쌍")


else:
    print(f"❌ 해결책을 찾지 못함 (상태: {solver.StatusName(status)})")
    print(f"  - 실제 시간: {solver.WallTime():.2f}초")


모델을 풀고 있습니다...
✅ 해결책 발견 (상태: OPTIMAL)
  - 실제 시간: 2.54초

✨ 최종 배정 결과 (총 학생 수: 200, 총 교실 수: 6)

[교실 0] (필요 정원: 33)
  - 배정 학생 수: 33 (✅ 정원 만족)
  - 리더십 학생 수: 4명 (✅ 리더십 ≥ 1)
  - 피아노 연주 학생 수: 1명
  - 남학생: 25명, 여학생: 8명
  - 성비: 남 75.76% / 여 24.24%
  - 운동 선호 학생 수: 5명
  - 클럽 분포: {'노래': 4, '댄스': 1, '야구': 2, '미술': 2, '밴드': 4, '축구': 2, '코딩': 5, '연극': 6, '봉사': 3, '독서': 4}
  - 싫어하는 관계 쌍 수: 0
  - 교실 0: 전년도 같은 반 학생 수 평균 = 5.50
  - 교실 0: 평균 점수 = 82.33
  - 교실 0 비등교 학생 수: 2명
  - 교실 0: 비등교-일반 학생 매칭 쌍 수 = 2쌍

[교실 1] (필요 정원: 33)
  - 배정 학생 수: 33 (✅ 정원 만족)
  - 리더십 학생 수: 5명 (✅ 리더십 ≥ 1)
  - 피아노 연주 학생 수: 5명
  - 남학생: 22명, 여학생: 11명
  - 성비: 남 66.67% / 여 33.33%
  - 운동 선호 학생 수: 5명
  - 클럽 분포: {'노래': 0, '댄스': 5, '야구': 5, '미술': 6, '밴드': 3, '축구': 2, '코딩': 4, '연극': 3, '봉사': 3, '독서': 2}
  - 싫어하는 관계 쌍 수: 0
  - 교실 1: 전년도 같은 반 학생 수 평균 = 5.50
  - 교실 1: 평균 점수 = 79.30
  - 교실 1 비등교 학생 수: 4명
  - 교실 1: 비등교-일반 학생 매칭 쌍 수 = 4쌍

[교실 2] (필요 정원: 33)
  - 배정 학생 수: 33 (✅ 정원 만족)
  - 리더십 학생 수: 2명 (✅ 리더십 ≥ 1)
  - 피아노 연주 학생 수: 3명
  - 남학생: 23명, 여학생:

In [19]:
df['new_class'] = df['id'].apply(lambda s_id: solver.Value(student_vars[int(s_id)]))

# --- CSV로 저장 ---
df.to_csv("/content/CP_result.csv", index=False, encoding="utf-8-sig")


✅ 최종 배정 결과를 '최적화된csv.csv' 파일로 저장했습니다.
